# Data Collection

### Objectives : 
1. Collecting city and country names by scaping data using BeautifulSoup from worldpopulationreview.com
2. Collecting respective geographical coordinates using OpenCageGeocode api
3. Collecting the nearby venues of each city using FourSquare api
4. Saving the data in disk for future use

#### Importing dependencies

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from opencage.geocoder import OpenCageGeocode
from geopy.exc import GeocoderTimedOut
import folium
import json


#### Requesting sorce page for city - country pairs

In [2]:
source = requests.get("http://worldpopulationreview.com/continents/cities-in-europe/").text

#### Collecting data using BeautifulSoup and OpenCageGeocode

In [54]:
geolocator = OpenCageGeocode('ea9636898f12432caccd5d49e4a11ec9')
def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [56]:
soup = BeautifulSoup(source, 'lxml')
table = soup.find('tbody')
data = []
for row in table.findAll('tr'):
    temp = []
    for element in row.findAll('td'):
        temp.append(element.text)
    location = do_geocode('{}, {}'.format(temp[0], temp[1]))
    temp.append(location[0]['geometry']['lat'])
    temp.append(location[0]['geometry']['lng'])
    data.append(temp)
df = pd.DataFrame(data, columns = ['City', 'Country', 'Population', 'Latitude', 'Longitude'])

#### Saving it in the disk

In [59]:
df.to_csv('europe_city_country.csv')

In [6]:
df = pd.read_csv('europe_city_country.csv', index_col=0)

#### Data Properties

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 5 columns):
City          500 non-null object
Country       500 non-null object
Population    500 non-null object
Latitude      500 non-null float64
Longitude     500 non-null float64
dtypes: float64(2), object(3)
memory usage: 23.4+ KB


In [7]:
df.head(10)

,City,Country,Population,Latitude,Longitude
0,Moscow,Russia,"10,381,222",55.750446,37.617494
1,London,United Kingdom,"7,556,900",51.507322,-0.127647
2,Saint Petersburg,Russia,"5,028,000",59.938732,30.316229
3,Berlin,Germany,"3,426,354",52.517036,13.388860
4,Madrid,Spain,"3,255,944",40.416705,-3.703582
5,Kyiv,Ukraine,"2,797,553",50.450034,30.524136
6,Rome,Italy,"2,318,895",41.894802,12.485338
7,Paris,France,"2,138,551",48.856697,2.351462
8,Bucharest,Romania,"1,877,155",44.436141,26.102720
9,Minsk,Belarus,"1,742,124",53.902334,27.561879


In [13]:
df.groupby(['Country'])['City'].count()

Country
Albania                     1
Austria                     6
Belarus                     8
Belgium                     5
Bosnia And Herzegovina      3
Bulgaria                    5
Croatia                     2
Cyprus                      1
Czech Republic              4
Denmark                     2
Estonia                     1
Finland                     5
France                     18
Germany                    60
Greece                      4
Hungary                     5
Ireland                     3
Italy                      19
Latvia                      1
Lithuania                   3
Macedonia                   1
Moldova                     2
Netherlands                10
Norway                      2
Poland                     25
Portugal                    3
Romania                    20
Russia                    127
Serbia                      3
Slovakia                    2
Slovenia                    1
Spain                      52
Sweden                      3
Sw

#### Collecting additional data about nearby venues using FourSquare api

In [138]:
Creds = pd.read_csv('FourSquareCredentials.csv')
CLIENT_ID = 'Creds['CLIENT_ID'][0]' # your Foursquare ID
CLIENT_SECRET = Creds['CLIENT_SECRET'][0] # your Foursquare Secret
VERSION = Creds['VERSION'][0] # Foursquare API version

In [84]:
radius = 10000
LIMIT = 100
def getNearbyVenues(names, countries, latitudes, longitudes):
    
    venues_list=[]
    for name, country, lat, lng in zip(names, countries, latitudes, longitudes):
        print('{}, {}'.format(name, country))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            country,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'Country',           
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [150]:
europe_venues = pd.read_csv('europe_venues.csv')

In [155]:
start = 0

In [149]:
parts = list(range(50, df.shape[0] + 1, 50))    

In [152]:
for end in parts:
    temp = df[start:end]
    temp_venues = getNearbyVenues(names=temp['City'],
                                   countries=temp['Country'],
                                   latitudes=temp['Latitude'],
                                   longitudes=temp['Longitude']
                                  )
    europe_venues = europe_venues.append(temp_venues, ignore_index = True)
    start = end
    print(start)

Torun, Poland
Kielce, Poland
Jerez de la Frontera, Spain
Oradea, Romania
Sabadell, Spain
Mostoles, Spain
Linz, Austria
Alcala de Henares, Spain
Ivano-Frankivsk, Ukraine
Debrecen, Hungary
Padova, Italy
Erfurt, Germany
Vasyl'evsky Ostrov, Russia
Tampere, Finland
Harburg, Germany
Pskov, Russia
Favoriten, Austria
Nicosia, Cyprus
Charleroi, Belgium
Severnyy, Russia
Tilburg, Netherlands
Balakovo, Russia
Armavir, Russia
Bila Tserkva, Ukraine
Dudley, United Kingdom
Hagen, Germany
Gliwice, Poland
Pamplona, Spain
Rostock, Germany
Fuenlabrada, Spain
Aberdeen, United Kingdom
Reims, France
Engel's, Russia
Burgas, Bulgaria
Kassel, Germany
Severodvinsk, Russia
Portsmouth, United Kingdom
Newcastle upon Tyne, United Kingdom
Klaipeda, Lithuania
Zabrze, Poland
Zlatoust, Russia
Cork, Ireland
Vantaa, Finland
Syzran', Russia
Bytom, Poland
Almeria, Spain
Praga Poludnie, Poland
Sutton, United Kingdom
Petropavlovsk-Kamchatsky, Russia
Trieste, Italy


C:\Users\soumy\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


400
Donaustadt, Austria
Leganes, Spain
Le Havre, France
Swindon, United Kingdom
Dun Laoghaire, Ireland
San Sebastian, Spain
Hamm, Germany
Mainz, Germany
Brescia, Italy
Geneve, Switzerland
Cergy-Pontoise, France
Korolyov, Russia
Sants-Montjuic, Spain
Santander, Spain
Liege, Belgium
Kamensk-Ural'skiy, Russia
Prato, Italy
Saarbruecken, Germany
Groningen, Netherlands
Taranto, Italy
Crawley, United Kingdom
Castello de la Plana, Spain
Yasenevo, Russia
Podolsk, Russia
Burgos, Spain
Amadora, Portugal
Ipswich, United Kingdom
Bielsko-Biala, Poland
Yuzhno-Sakhalinsk, Russia
Almere Stad, Netherlands
Split, Croatia
Saint-Etienne, France
Turku, Finland
Wigan, United Kingdom
Kramators'k, Ukraine
Croydon, United Kingdom
Miskolc, Hungary
Walsall, United Kingdom
Herne, Germany
Lyublino, Russia
Mansfield, United Kingdom
Olsztyn, Poland
Bacau, Romania
Oxford, United Kingdom
Muelheim, Germany
Albacete, Spain
Reggio Calabria, Italy
Arad, Romania
Baranovichi, Belarus
Toulon, France
450
Angers, France
Horta-G

In [160]:
europe_venues.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace=True)

#### Saving it in the disk for future use

In [162]:
europe_venues.to_csv('europe_venues.csv')

In [17]:
europe_venues = pd.read_csv('europe_venues.csv', index_col=0)

#### Data properties

In [18]:
europe_venues.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43658 entries, 0 to 43657
Data columns (total 8 columns):
City               43658 non-null object
City Latitude      43658 non-null float64
City Longitude     43658 non-null float64
Country            43658 non-null object
Venue              43658 non-null object
Venue Category     43658 non-null object
Venue Latitude     43658 non-null float64
Venue Longitude    43658 non-null float64
dtypes: float64(4), object(4)
memory usage: 3.0+ MB


In [19]:
europe_venues.sample(10)

,City,City Latitude,City Longitude,Country,Venue,Venue Category,Venue Latitude,Venue Longitude
35636,Swindon,51.561368,-1.785685,United Kingdom,Costa Coffee,Coffee Shop,51.560484,-1.784737
38836,Lyublino,55.675638,37.761806,Russia,Reserved,Clothing Store,55.654851,37.844804
5324,Amsterdam,52.374540,4.897975,Netherlands,De Laatste Kruimel,Bakery,52.369220,4.892810
34728,Vantaa,60.309187,25.036453,Finland,Rönttösrouva,Bakery,60.262762,25.026598
31873,Ivano-Frankivsk,48.922522,24.710319,Ukraine,Пивний клуб «Десятка» / Beer Club 10 (Пивний к...,Pub,48.919870,24.711959
16883,Sochi,43.585482,39.723109,Russia,Че? Харчо!,Restaurant,43.577009,39.722460
41156,Pilsen,49.747741,13.377525,Czech Republic,Supa Supa,Soup Place,49.747166,13.379010
27997,Ciudad Lineal,40.448621,-3.654835,Spain,Auditorio Nacional de Música,Concert Hall,40.446068,-3.677757
25457,A Coruna,43.371209,-8.395877,Spain,Jamonería La Leonesa,Spanish Restaurant,43.369916,-8.392296
40841,Warrington,53.401858,-2.568022,United Kingdom,Haydock Park Racecourse,Racecourse,53.480440,-2.626643


In [21]:
europe_venues.groupby(['Country']).count()['Venue']

Country
Albania                    100
Austria                    600
Belarus                    618
Belgium                    500
Bosnia And Herzegovina     177
Bulgaria                   490
Croatia                    200
Cyprus                     100
Czech Republic             400
Denmark                    200
Estonia                    100
Finland                    500
France                    1572
Germany                   5635
Greece                     400
Hungary                    485
Ireland                    300
Italy                     1798
Latvia                     100
Lithuania                  257
Macedonia                  100
Moldova                    160
Netherlands               1000
Norway                     200
Poland                    1916
Portugal                   300
Romania                   1682
Russia                    9429
Serbia                     279
Slovakia                   200
Slovenia                   100
Spain                     4940
